In [1]:
import time
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.profiler as profiler
from torch.profiler import ProfilerActivity, profile, record_function

# Задание 1: Выбор предобученной модели

### 1.1. Выбор и загрузка модели

Для выполнения задания выберем модель `sberbank-ai/rugpt3small_based_on_gpt2`. 

Это русскоязычная GPT-модель с 125 миллионами параметров, что соответствует условию (до 1B параметров) и хорошо подходит для экспериментов с профилированием на ограниченных ресурсах.

In [2]:
MODEL_NAME = "sberbank-ai/rugpt3small_based_on_gpt2"
LOCAL_MODEL_PATH = "./rugpt3small_local"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def load_or_get_model(model_name, local_path, device):
    if "model" in globals() and "tokenizer" in globals():
        print("Модель уже в памяти, переиспользую без перезагрузки.")
        model.to(device)
        return tokenizer, model

    if os.path.isdir(local_path):
        print(f"Загружаю модель из локальной копии: {local_path}...")
        tokenizer = AutoTokenizer.from_pretrained(local_path, local_files_only=True)
        model = AutoModelForCausalLM.from_pretrained(local_path, local_files_only=True)
    else:
        print(f"Локальная копия модели не найдена. Скачиваю {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        print(f"Сохраняю модель в {local_path}...")
        model.save_pretrained(local_path)
        tokenizer.save_pretrained(local_path)
        print("Модель успешно сохранена.")

    model.to(device)
    model.eval()
    model.config.use_cache = True

    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id

    print(f"Модель и токенизатор успешно загружены на {device}.")
    print(f"Количество параметров: {model.num_parameters() / 1e6:.1f}M")

    return tokenizer, model


try:
    tokenizer, model = load_or_get_model(MODEL_NAME, LOCAL_MODEL_PATH, DEVICE)
except Exception as e:
    print(f"Произошла ошибка: {e}")


Загружаю модель из локальной копии: ./rugpt3small_local...
Модель и токенизатор успешно загружены на cuda.
Количество параметров: 125.2M


# Задание 2: Базовый прогон (финальная, корректная версия)

In [3]:
@torch.inference_mode()
def measure_performance(input_ids, max_new_tokens):
    # --- Prefill ---
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    t0 = time.perf_counter()

    # Прямой вызов модели для prefill'а, получаем KV-кэш
    out = model(input_ids=input_ids, use_cache=True)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    prefill_time = time.perf_counter() - t0

    # --- Decode ---
    past = out.past_key_values
    # Берем последний токен для начала декодирования
    next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    decode_tokens = max(max_new_tokens - 1, 0)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    t1 = time.perf_counter()

    # Цикл декодирования
    for _ in range(decode_tokens):
        out = model(input_ids=next_token, past_key_values=past, use_cache=True)
        past = out.past_key_values
        next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    decode_time = time.perf_counter() - t1
    decode_time_per_token = decode_time / decode_tokens if decode_tokens > 0 else 0

    # --- Расчет метрик ---
    e2e_time = prefill_time + decode_time
    tps = decode_tokens / decode_time if decode_time > 0 else 0

    return {
        "TTFT (prefill, c)": prefill_time,
        "Decode (c)": decode_time,
        "Decode per token (c)": decode_time_per_token,
        "E2E (c)": e2e_time,
        "TPS (ток/с)": tps,
    }


L_values = [512, 1024, 1536]
max_new_tokens = 128
num_iterations = 10
all_results = []

base_prompt = "Искусственный интеллект и машинное обучение революционизировали современные технологии. "
base_tokens = tokenizer.encode(base_prompt)

if DEVICE == "cuda":
    print("Прогрев GPU...")
    warm_inputs = tokenizer(base_prompt, return_tensors="pt").to(DEVICE)
    _ = measure_performance(warm_inputs["input_ids"], 16)
    print("Прогрев завершен.")

for L in L_values:
    print(f"\n--- Измерения для L = {L} (усреднение по {num_iterations} итерациям) ---")
    iter_results = []

    repeat_count = max(1, L // max(1, len(base_tokens)))
    prompt = base_prompt * repeat_count
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=L).to(DEVICE)
    input_ids = inputs["input_ids"]

    for i in range(num_iterations):
        if DEVICE == "cuda":
            torch.cuda.reset_peak_memory_stats()
            base_alloc = torch.cuda.memory_allocated()

        perf_metrics = measure_performance(input_ids, max_new_tokens)

        if DEVICE == "cuda":
            peak_alloc = torch.cuda.max_memory_allocated()
            # Пиковая память сверх базовой (весов модели), чтобы видеть вклад KV-кэша
            perf_metrics["VRAM (MB)"] = (peak_alloc - base_alloc) / (1024**2)
        else:
            perf_metrics["VRAM (MB)"] = 0
        iter_results.append(perf_metrics)

    avg_results = pd.DataFrame(iter_results).mean().to_dict()
    avg_results["L (вход)"] = L
    all_results.append(avg_results)
    print(f"Результаты для L={L} сохранены.")

# --- Отображение результатов ---
df_results = pd.DataFrame(all_results)
df_results = df_results[
    ["L (вход)", "VRAM (MB)", "TTFT (prefill, c)", "E2E (c)", "Decode (c)", "Decode per token (c)", "TPS (ток/с)"]
]
df_results = df_results.round(4)


Прогрев GPU...
Прогрев завершен.

--- Измерения для L = 512 (усреднение по 10 итерациям) ---
Результаты для L=512 сохранены.

--- Измерения для L = 1024 (усреднение по 10 итерациям) ---
Результаты для L=1024 сохранены.

--- Измерения для L = 1536 (усреднение по 10 итерациям) ---
Результаты для L=1536 сохранены.


In [4]:
display(df_results)

,L (вход),VRAM (MB),"TTFT (prefill, c)",E2E (c),Decode (c),Decode per token (c),TPS (ток/с)
0,512,128.2168,0.0211,1.3486,1.3274,0.0105,96.1517
1,1024,238.2891,0.0388,1.2679,1.2291,0.0097,103.7283
2,1536,358.3906,0.0594,1.2990,1.2396,0.0098,102.7924


### Интерпретация результатов (фактические наблюдения)

По текущим измерениям (L = 512/1024/1536, max_new_tokens = 128) видны следующие тренды:

1. **VRAM** растет почти линейно с увеличением `L` — это ожидаемо из-за роста KV-кэша.
2. **TTFT / prefill** монотонно растет с `L`, что соответствует росту стоимости внимания на полном контексте.
3. **Decode time** и **TPS** почти не меняются (колебания в пределах шума). Это означает, что в данном диапазоне длин и для этой модели вклад внимания в шаге decode относительно мал по сравнению с постоянными затратами (MLP, argmax, оверхед запуска).
4. **E2E** меняется слабо и в основном определяется суммой почти постоянного decode и умеренно растущего prefill.

Итог: теоретически decode должен расти с `L`, но для небольшой модели и выбранных параметров эффект оказывается слабым и теряется в шуме. Чтобы усилить тренд, нужны большее `L`, больше `max_new_tokens` и/или более крупная модель.


# Задание 3: Профилирование модели (PyTorch Profiler + TensorBoard)


In [5]:
import os
import pandas as pd
import torch
import torch.profiler as profiler
from torch.profiler import ProfilerActivity, profile, record_function

LOGDIR = "./tb_profiler"
os.makedirs(LOGDIR, exist_ok=True)

base_prompt = "Искусственный интеллект и машинное обучение революционизировали современные технологии. "


def build_inputs(target_len):
    base_tokens = tokenizer.encode(base_prompt)
    repeat_count = max(1, target_len // max(1, len(base_tokens)))
    prompt = base_prompt * repeat_count
    return tokenizer(prompt, return_tensors="pt", truncation=True, max_length=target_len).to(DEVICE)


@torch.inference_mode()
def run_prefill_decode(inputs, decode_steps):
    with record_function("prefill"):
        out = model(**inputs, use_cache=True)
    past = out.past_key_values
    next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    with record_function("decode"):
        for _ in range(decode_steps):
            out = model(input_ids=next_token, past_key_values=past, use_cache=True)
            past = out.past_key_values
            next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)


def profile_scenario(name, inputs, decode_steps):
    activities = [ProfilerActivity.CPU]
    if torch.cuda.is_available():
        activities.append(ProfilerActivity.CUDA)

    with profile(
        activities=activities,
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        on_trace_ready=profiler.tensorboard_trace_handler(os.path.join(LOGDIR, name)),
    ) as prof:
        run_prefill_decode(inputs, decode_steps)

    return prof


def top_ops_by_time(prof, top_n=5):
    def pick_attr(event, names, default=None):
        for name in names:
            try:
                value = getattr(event, name)
            except AttributeError:
                continue
            if value is not None:
                return value
        return default

    def has_attr(event, names):
        for name in names:
            try:
                getattr(event, name)
            except AttributeError:
                continue
            else:
                return True
        return False

    events = prof.key_averages()

    # Prefer device_time to avoid deprecated cuda_time.
    use_device_time = any(has_attr(e, ["device_time_total", "device_time"]) for e in events)

    if use_device_time:
        get_time = lambda e: pick_attr(e, ["device_time_total", "device_time"], 0.0)
        get_self = lambda e: pick_attr(e, ["self_device_time_total", "self_device_time"], 0.0)
        time_label = "device_time_ms"
        self_label = "self_device_time_ms"
    else:
        get_time = lambda e: pick_attr(e, ["cpu_time_total", "cpu_time"], 0.0)
        get_self = lambda e: pick_attr(e, ["self_cpu_time_total", "self_cpu_time"], 0.0)
        time_label = "cpu_time_ms"
        self_label = "self_cpu_time_ms"

    total = sum(get_time(e) for e in events)

    rows = []
    for e in sorted(events, key=get_time, reverse=True)[:top_n]:
        rows.append(
            {
                "op": e.key,
                time_label: get_time(e) / 1000,
                self_label: get_self(e) / 1000,
                "calls": e.count,
                "share_%": (get_time(e) / total * 100) if total else 0.0,
            }
        )
    return pd.DataFrame(rows)


In [6]:
# Сценарий 1: prefill-доминантный (длинный вход, короткая генерация)
prefill_inputs = build_inputs(1024)
prof_prefill = profile_scenario("prefill_dominant", prefill_inputs, decode_steps=8)
top_prefill = top_ops_by_time(prof_prefill)
display(top_prefill)

# Сценарий 2: decode-доминантный (короткий вход, длинная генерация)
decode_inputs = build_inputs(16)
prof_decode = profile_scenario("decode_dominant", decode_inputs, decode_steps=128)
top_decode = top_ops_by_time(prof_decode)
display(top_decode)


,op,device_time_ms,self_device_time_ms,calls,share_%
0,decode,178.787325,178.787325,1,34.063724
1,prefill,61.007864,61.007864,1,11.623615
2,prefill,37.918572,0.000000,1,7.224493
3,decode,34.264543,0.000000,1,6.528304
4,aten::addmm,25.735497,25.735497,432,4.903294


,op,device_time_ms,self_device_time_ms,calls,share_%
0,decode,2303.829400,2303.829400,1,58.528765
1,decode,435.116814,0.000000,1,11.054139
2,aten::addmm,216.249398,216.226526,6192,5.493814
3,"std::enable_if<!(false), void>::type internal:...",79.992708,79.992708,3072,2.032214
4,aten::scaled_dot_product_attention,70.607965,0.000000,1548,1.793795


In [7]:
%load_ext tensorboard
%tensorboard --logdir ./tb_profiler


### Интерпретация профилирования

**Сценарий prefill-dominant (L≈1024, decode_steps=8), top-5 CUDA:**
- decode — 176.37 ms (51.04%)
- prefill — 83.72 ms (24.23%)
- prefill — 37.82 ms (10.94%)
- decode — 34.90 ms (10.10%)
- ampere_sgemm_128x64_tn — 6.94 ms (2.01%)

**Сценарий decode-dominant (L≈16, decode_steps=128), top-5 CUDA:**
- decode — 2318.71 ms (80.40%)
- decode — 533.76 ms (18.51%)
- prefill — 21.70 ms (0.75%)
- prefill — 5.72 ms (0.20%)
- ampere_sgemm_64x32_sliced1x4_tn — 0.88 ms (0.03%)

**Выводы по логам:**
1. Большая часть времени находится внутри регионов `prefill`/`decode`, а отдельные GEMM-ядра дают малую долю. Это означает, что время распределено по множеству мелких операций, а не одной крупной матрице. Явных HtoD/DtoH копирований в top-5 нет.
2. По этой картине сценарии ближе к memory/overhead-bound (много мелких ops и запусков), особенно на decode. Для compute-bound ожидался бы доминирующий вклад крупных GEMM/attention.
3. Prefill vs decode: в decode-dominant режиме почти все время уходит на decode (~99%). В prefill-dominant режиме decode все равно занимает заметную долю (~61%), что объясняется несколькими шагами decode и небольшим batch.
4. Релевантные оптимизации: для prefill — FlashAttention/SDPA и увеличение batch/sequence; для decode — оптимизация KV-кэша (paged KV), kernel fusion (attention/MLP/layernorm), speculative decoding или батчинг запросов, чтобы снизить per-token overhead.


# Задание 4: Эксперименты с квантизованной моделью (4-bit)

> Примечание: перед запуском задания 4 надо перезапустить ядро интерпретатора и выполнить все ячейки до конца раздела 1. Это очистит память и обеспечит корректные замеры.


In [3]:
import gc
from transformers import BitsAndBytesConfig


def cleanup_model(m):
    del m
    gc.collect()
    if DEVICE == "cuda":
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.synchronize()


def get_vram_stats_mb():
    if DEVICE != "cuda":
        return {"vram_allocated_mb": 0.0, "vram_reserved_mb": 0.0}
    torch.cuda.synchronize()
    return {
        "vram_allocated_mb": torch.cuda.memory_allocated() / (1024**2),
        "vram_reserved_mb": torch.cuda.memory_reserved() / (1024**2),
    }


def load_tokenizer_safe(model_name, local_path):
    if "tokenizer" in globals():
        return tokenizer
    local_only = os.path.isdir(local_path)
    src = local_path if local_only else model_name
    kwargs = {"local_files_only": True} if local_only else {}
    return AutoTokenizer.from_pretrained(src, **kwargs)


def load_fp16_model(model_name, local_path):
    if DEVICE != "cuda":
        raise RuntimeError("FP16/4-bit эксперименты требуют CUDA.")

    local_only = os.path.isdir(local_path)
    src = local_path if local_only else model_name
    kwargs = {"local_files_only": True} if local_only else {}

    model_fp16 = AutoModelForCausalLM.from_pretrained(
        src,
        dtype=torch.float16,
        device_map="auto",
        **kwargs,
    )
    model_fp16.eval()
    model_fp16.config.use_cache = True
    return model_fp16


def load_4bit_model(model_name, local_path):
    if DEVICE != "cuda":
        raise RuntimeError("4-bit эксперименты требуют CUDA.")

    local_only = os.path.isdir(local_path)
    src = local_path if local_only else model_name
    kwargs = {"local_files_only": True} if local_only else {}

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    model_4bit = AutoModelForCausalLM.from_pretrained(
        src,
        quantization_config=bnb_config,
        device_map="auto",
        **kwargs,
    )
    model_4bit.eval()
    model_4bit.config.use_cache = True
    return model_4bit


if "MODEL_NAME" not in globals() or "LOCAL_MODEL_PATH" not in globals():
    raise RuntimeError("Сначала выполните раздел 1 (MODEL_NAME/LOCAL_MODEL_PATH).")

tokenizer = load_tokenizer_safe(MODEL_NAME, LOCAL_MODEL_PATH)

for name in ("model", "model_fp16", "model_4bit"):
    if name in globals():
        cleanup_model(globals()[name])
        globals().pop(name, None)

model_fp16 = load_fp16_model(MODEL_NAME, LOCAL_MODEL_PATH)
vram_fp16 = get_vram_stats_mb()
cleanup_model(model_fp16)

model_4bit = load_4bit_model(MODEL_NAME, LOCAL_MODEL_PATH)
vram_4bit = get_vram_stats_mb()

df_vram_compare = pd.DataFrame(
    [
        {"precision": "fp16", **vram_fp16},
        {"precision": "4-bit", **vram_4bit},
    ]
)
display(df_vram_compare)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_4bit.config.pad_token_id = model_4bit.config.eos_token_id


,precision,vram_allocated_mb,vram_reserved_mb
0,fp16,291.737305,644.0
1,4-bit,462.374023,586.0


In [4]:
@torch.inference_mode()
def measure_performance_with_model(model, input_ids, max_new_tokens):
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    t0 = time.perf_counter()

    out = model(input_ids=input_ids, use_cache=True)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    prefill_time = time.perf_counter() - t0

    past = out.past_key_values
    next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    decode_tokens = max(max_new_tokens - 1, 0)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    t1 = time.perf_counter()

    for _ in range(decode_tokens):
        out = model(input_ids=next_token, past_key_values=past, use_cache=True)
        past = out.past_key_values
        next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    if DEVICE == "cuda":
        torch.cuda.synchronize()
    decode_time = time.perf_counter() - t1
    decode_time_per_token = decode_time / decode_tokens if decode_tokens > 0 else 0

    e2e_time = prefill_time + decode_time
    tps = decode_tokens / decode_time if decode_time > 0 else 0

    return {
        "TTFT (prefill, c)": prefill_time,
        "Decode (c)": decode_time,
        "Decode per token (c)": decode_time_per_token,
        "E2E (c)": e2e_time,
        "TPS (ток/с)": tps,
    }


L_values_4bit = [512, 1024, 1536]
max_new_tokens_4bit = 128
num_iterations_4bit = 10
all_results_4bit = []

base_prompt = "Искусственный интеллект и машинное обучение революционизировали современные технологии. "
base_tokens = tokenizer.encode(base_prompt)

if DEVICE == "cuda":
    print("Прогрев GPU (4-bit)...")
    warm_inputs = tokenizer(base_prompt, return_tensors="pt").to(DEVICE)
    _ = measure_performance_with_model(model_4bit, warm_inputs["input_ids"], 16)
    print("Прогрев завершен.")

for L in L_values_4bit:
    print(f"\n--- Измерения для L = {L} (4-bit, усреднение по {num_iterations_4bit} итерациям) ---")
    iter_results = []

    repeat_count = max(1, L // max(1, len(base_tokens)))
    prompt = base_prompt * repeat_count
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=L).to(DEVICE)
    input_ids = inputs["input_ids"]

    for _ in range(num_iterations_4bit):
        if DEVICE == "cuda":
            torch.cuda.reset_peak_memory_stats()
            base_alloc = torch.cuda.memory_allocated()

        perf_metrics = measure_performance_with_model(model_4bit, input_ids, max_new_tokens_4bit)

        if DEVICE == "cuda":
            peak_alloc = torch.cuda.max_memory_allocated()
            perf_metrics["VRAM (MB)"] = (peak_alloc - base_alloc) / (1024**2)
        else:
            perf_metrics["VRAM (MB)"] = 0

        iter_results.append(perf_metrics)

    avg_results = pd.DataFrame(iter_results).mean().to_dict()
    avg_results["L (вход)"] = L
    all_results_4bit.append(avg_results)
    print(f"Результаты для L={L} сохранены.")

df_results_4bit = pd.DataFrame(all_results_4bit)
df_results_4bit = df_results_4bit[
    ["L (вход)", "VRAM (MB)", "TTFT (prefill, c)", "E2E (c)", "Decode (c)", "Decode per token (c)", "TPS (ток/с)"]
]
df_results_4bit = df_results_4bit.round(4)
display(df_results_4bit)


Прогрев GPU (4-bit)...
Прогрев завершен.

--- Измерения для L = 512 (4-bit, усреднение по 10 итерациям) ---
Результаты для L=512 сохранены.

--- Измерения для L = 1024 (4-bit, усреднение по 10 итерациям) ---
Результаты для L=1024 сохранены.

--- Измерения для L = 1536 (4-bit, усреднение по 10 итерациям) ---
Результаты для L=1536 сохранены.


,L (вход),VRAM (MB),"TTFT (prefill, c)",E2E (c),Decode (c),Decode per token (c),TPS (ток/с)
0,512,59.6680,0.0297,2.0459,2.0162,0.0159,63.0259
1,1024,120.8652,0.0406,2.1359,2.0953,0.0165,60.6804
2,1536,179.3384,0.0480,2.0892,2.0411,0.0161,62.2450


### Интерпретация 4-bit результатов

1. **VRAM (веса модели)**: по `df_vram_compare` получено FP16 ≈ 291.74 MB allocated и 644.0 MB reserved, а 4-bit ≈ 462.37 MB allocated и 586.0 MB reserved. Это не «чистый размер весов», а состояние CUDA‑аллокатора с буферами и метаданными квантования, поэтому значения читаем аккуратно.
2. **VRAM (динамика во время инференса)**: пиковая прибавка памяти на KV‑кэш заметно ниже в 4-bit: ~59.7 / 120.9 / 179.3 MB против ~128.2 / 238.3 / 358.4 MB в FP16 (примерно в 2 раза меньше на всех L).
3. **TTFT/Decode/TPS**: TTFT сопоставим по порядку (≈0.03–0.05 c), но decode и E2E в 4-bit ощутимо медленнее (≈2.04–2.14 c против ≈1.27–1.35 c), TPS ниже (≈60–63 ток/с против ≈96–104 ток/с).
4. **Prefill vs decode**: в обеих версиях доминирует decode, но в 4-bit он еще сильнее определяет итоговое E2E‑время.


In [5]:
LOGDIR_4BIT = "./tb_profiler_4bit"
os.makedirs(LOGDIR_4BIT, exist_ok=True)


def build_inputs_4bit(target_len, prompt_text):
    base_tokens = tokenizer.encode(prompt_text)
    repeat_count = max(1, target_len // max(1, len(base_tokens)))
    prompt = prompt_text * repeat_count
    return tokenizer(prompt, return_tensors="pt", truncation=True, max_length=target_len).to(DEVICE)


@torch.inference_mode()
def run_prefill_decode_4bit(model, inputs, decode_steps):
    with record_function("prefill"):
        out = model(**inputs, use_cache=True)
    past = out.past_key_values
    next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)

    with record_function("decode"):
        for _ in range(decode_steps):
            out = model(input_ids=next_token, past_key_values=past, use_cache=True)
            past = out.past_key_values
            next_token = out.logits[:, -1, :].argmax(dim=-1, keepdim=True)


def profile_scenario_4bit(name, model, inputs, decode_steps, logdir):
    activities = [ProfilerActivity.CPU]
    if torch.cuda.is_available():
        activities.append(ProfilerActivity.CUDA)

    with profile(
        activities=activities,
        record_shapes=True,
        profile_memory=True,
        with_stack=True,
        on_trace_ready=profiler.tensorboard_trace_handler(os.path.join(logdir, name)),
    ) as prof:
        run_prefill_decode_4bit(model, inputs, decode_steps)

    return prof


def top_ops_by_time_4bit(prof, top_n=5):
    def pick_attr(event, names, default=None):
        for name in names:
            try:
                value = getattr(event, name)
            except AttributeError:
                continue
            if value is not None:
                return value
        return default

    def has_attr(event, names):
        for name in names:
            try:
                getattr(event, name)
            except AttributeError:
                continue
            else:
                return True
        return False

    events = prof.key_averages()

    use_device_time = any(has_attr(e, ["device_time_total", "device_time"]) for e in events)

    if use_device_time:
        get_time = lambda e: pick_attr(e, ["device_time_total", "device_time"], 0.0)
        get_self = lambda e: pick_attr(e, ["self_device_time_total", "self_device_time"], 0.0)
        time_label = "device_time_ms"
        self_label = "self_device_time_ms"
    else:
        get_time = lambda e: pick_attr(e, ["cpu_time_total", "cpu_time"], 0.0)
        get_self = lambda e: pick_attr(e, ["self_cpu_time_total", "self_cpu_time"], 0.0)
        time_label = "cpu_time_ms"
        self_label = "self_cpu_time_ms"

    total = sum(get_time(e) for e in events)

    rows = []
    for e in sorted(events, key=get_time, reverse=True)[:top_n]:
        rows.append(
            {
                "op": e.key,
                time_label: get_time(e) / 1000,
                self_label: get_self(e) / 1000,
                "calls": e.count,
                "share_%": (get_time(e) / total * 100) if total else 0.0,
            }
        )
    return pd.DataFrame(rows)


prefill_inputs_4bit = build_inputs_4bit(1024, base_prompt)
prof_prefill_4bit = profile_scenario_4bit("prefill_dominant_4bit", model_4bit, prefill_inputs_4bit, 8, LOGDIR_4BIT)
top_prefill_4bit = top_ops_by_time_4bit(prof_prefill_4bit)
display(top_prefill_4bit)

decode_inputs_4bit = build_inputs_4bit(16, base_prompt)
prof_decode_4bit = profile_scenario_4bit("decode_dominant_4bit", model_4bit, decode_inputs_4bit, 128, LOGDIR_4BIT)
top_decode_4bit = top_ops_by_time_4bit(prof_decode_4bit)
display(top_decode_4bit)


,op,device_time_ms,self_device_time_ms,calls,share_%
0,decode,318.126867,318.126867,1,50.190186
1,prefill,60.935774,60.935774,1,9.613705
2,prefill,34.914074,0.000000,1,5.508318
3,aten::linear,26.412399,0.000000,57,4.167027
4,decode,26.150154,0.000000,1,4.125653


,op,device_time_ms,self_device_time_ms,calls,share_%
0,decode,5084.938689,5084.938689,1,55.352779
1,decode,1003.200173,0.000000,1,10.920469
2,aten::linear,348.983518,0.000000,177,3.798907
3,aten::matmul,329.214299,0.000000,129,3.583706
4,aten::mm,329.214299,329.214299,129,3.583706


In [6]:
%load_ext tensorboard
%tensorboard --logdir ./tb_profiler_4bit


### Интерпретация профилирования 4-bit

**Prefill-dominant (L≈1024, decode_steps=8), top-5 device_time:**
- decode — 318.13 ms (50.19%)
- prefill — 60.94 ms (9.61%)
- prefill — 34.91 ms (5.51%)
- aten::linear — 26.41 ms (4.17%)
- decode — 26.15 ms (4.13%)

**Decode-dominant (L≈16, decode_steps=128), top-5 device_time:**
- decode — 5084.94 ms (55.35%)
- decode — 1003.20 ms (10.92%)
- aten::linear — 348.98 ms (3.80%)
- aten::matmul — 329.21 ms (3.58%)
- aten::mm — 329.21 ms (3.58%)

**Выводы:**
1. В обоих сценариях лидирует `decode`, а в top‑5 заметны `aten::linear/matmul/mm`, что указывает на вклад матмул‑операций и де‑квантного оверхеда.
2. В decode‑dominant режиме почти все время уходит на токен‑токен декодирование (две строки `decode` дают ≈66% суммарного времени).
3. Явных HtoD/DtoH копирований в top‑5 нет — узкое место не в трансферах, а в вычислениях и оверхеде мелких ops.
4. По сравнению с FP16 4-bit выглядит более «overhead/memory‑bound»: больше отдельных linear/matmul в топе и слабее выраженный вклад attention.
